In [0]:
import numpy as np
import pandas as pd
import math
from pathlib import Path
from torch.optim.lr_scheduler import OneCycleLR
import os
import random
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms
from fastprogress import master_bar, progress_bar
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
from torch.utils.data.sampler import Sampler
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
import PIL


In [0]:
from pyronear.datasets.wildfire import (WildFireDataset,
                                        WildFireSplitter,FireLabeler)
from pyronear.datasets.wildfire.split_strategy import *

In [0]:
torch.cuda.empty_cache()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
  import os
os.chdir("drive/My Drive/DeepLearning/Pyronear/git/PyroNear")
!ls

CONTRIBUTING.md  LICENSE    references	      test
CreateDS.ipynb	 pyronear   requirements.txt  TrainWildFire.ipynb
docs		 README.md  setup.py	      TrainWildFire_SubSampler.ipynb


In [5]:
os.chdir("..")
os.chdir("..")
!ls


 Clean_checkedLabels.ipynb   GetFrame.ipynb		   TrainModel.ipynb
 CreateDataset.ipynb	     git			   TrainModelToy.ipynb
 customDSTest.ipynb	     PR				   TrainOldDS.ipynb
 DB			    'TrainFastAi[Conflit].ipynb'   TrainOldDSToy.ipynb
 dfToy.csv		     TrainFastAi.ipynb		   TrainWild.ipynb
 dfToyEasy.csv		     TrainFGLoop2.ipynb		   videoanalysis
 dict2.txt		     TrainFGLoop_bce.ipynb	   WF.csv
 dict_Old.txt		     TrainFGLoop.ipynb		   wildfire.csv
 dict.txt		     TrainFGLoop_old.ipynb	   WildfireDS.csv
 dict_wild.txt		     TrainFGLoop_oldToy.ipynb	  'WildFire DS.ipynb'
 EasyDS.ipynb		    'TrainModel[Conflit].ipynb'


In [7]:
path_to_frames=Path("DB/Wildfire")
metadata=pd.read_csv("DB/Wildfire/metadata2.csv")
metadata

,Unnamed: 0,Unnamed: 0.1,exploitable,fire,sequence,clf_confidence,loc_confidence,x,y,t,stateStart,stateEnd,frame,imgFile,fire_id,fBase
0,0,0,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,14,108_frame14.png,20,108
1,1,1,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,71,108_frame71.png,20,108
2,2,2,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,92,108_frame92.png,20,108
3,3,3,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,102,108_frame102.png,20,108
4,4,4,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,106,108_frame106.png,20,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10830,10955,10955,True,True,0,False,True,483.725,341.641,2.353,1333.0,1501.0,1347,86_frame1347.png,164,86
10831,10956,10956,True,True,0,False,True,483.725,341.641,2.353,1333.0,1501.0,1404,86_frame1404.png,164,86
10832,10957,10957,True,True,0,False,True,483.725,341.641,2.353,1333.0,1501.0,1425,86_frame1425.png,164,86
10833,10958,10958,True,True,0,False,True,483.725,341.641,2.353,1333.0,1501.0,1435,86_frame1435.png,164,86


In [8]:
metadata=metadata[metadata['clf_confidence']==True]
metadata

,Unnamed: 0,Unnamed: 0.1,exploitable,fire,sequence,clf_confidence,loc_confidence,x,y,t,stateStart,stateEnd,frame,imgFile,fire_id,fBase
0,0,0,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,14,108_frame14.png,20,108
1,1,1,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,71,108_frame71.png,20,108
2,2,2,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,92,108_frame92.png,20,108
3,3,3,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,102,108_frame102.png,20,108
4,4,4,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,106,108_frame106.png,20,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10810,10935,10935,True,False,0,True,True,353.110,389.282,1.484,37.0,1328.0,897,758_frame897.png,100,758
10811,10936,10936,True,False,0,True,True,353.110,389.282,1.484,37.0,1328.0,1081,758_frame1081.png,100,758
10812,10937,10937,True,False,0,True,True,353.110,389.282,1.484,37.0,1328.0,1132,758_frame1132.png,100,758
10813,10938,10938,True,False,0,True,True,353.110,389.282,1.484,37.0,1328.0,1163,758_frame1163.png,100,758


In [9]:
#equalize dataset
ratio=len(metadata[metadata['fire']==True])/len(metadata[metadata['fire']==False])
ratio

1.9838449111470113

In [10]:
metadata1=metadata[0:len(metadata):5]
metadata2=metadata[3:len(metadata):5]
metadata2=metadata2[metadata2['fire']==False]
metadata=pd.concat([metadata1,metadata2])
metadata.index=np.arange(len(metadata))
metadata

,Unnamed: 0,Unnamed: 0.1,exploitable,fire,sequence,clf_confidence,loc_confidence,x,y,t,stateStart,stateEnd,frame,imgFile,fire_id,fBase
0,0,0,True,True,0,True,True,742.268,417.155,0.005,0.0,151.0,14,108_frame14.png,20,108
1,10,10,True,True,0,True,2,817.237,496.577,3.683,197.0,1500.0,1057,110_frame1057.png,171,110
2,15,15,True,True,0,True,True,585.649,514.392,0.087,2.0,1453.0,862,115_frame862.png,19,115
3,20,20,True,True,0,True,True,230.103,388.948,0.000,0.0,145.0,14,118_frame14.png,19,118
4,25,25,True,True,0,True,True,739.299,304.330,0.000,146.0,158.0,149,118_frame149.png,19,118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,10883,10883,True,False,0,True,True,664.880,380.670,1.580,39.0,1485.0,1169,635_frame1169.png,346,635
2463,10888,10888,True,False,0,True,True,738.947,328.995,11.548,289.0,1499.0,1415,640_frame1415.png,351,640
2464,10903,10903,True,False,0,True,True,1005.072,545.167,3.225,81.0,102.0,95,695_frame95.png,376,695
2465,10913,10913,True,False,0,True,True,565.837,454.737,1.980,1380.0,1454.0,1440,695_frame1440.png,376,695


In [0]:
metadata.loc[metadata['fire']==True,'fire']=1
metadata.loc[metadata['fire']==False,'fire']=0

In [12]:
ratio=len(metadata[metadata['fire']==1])/len(metadata[metadata['fire']==0])
ratio

0.9911218724778047

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    normalize
])

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
   normalize
])

tf={'train':train_transforms,'test':train_transforms,'val':train_transforms}

In [0]:
wildfire = WildFireDataset(metadata=metadata,path_to_frames=path_to_frames,target_names=['fire'])

In [15]:
ratios = {'train': 0.78, 'val': 0.2, 'test':0.02}

splitter =  WildFireSplitter(ratios, transforms=tf)
splitter.fit(wildfire)
print(splitter.n_samples_)

train_loader = DataLoader(splitter.train, batch_size=64, shuffle=True)
val_loader = DataLoader(splitter.val, batch_size=64, shuffle=True)
test_loader= DataLoader(splitter.test, batch_size=64, shuffle=True)

{'train': 1941, 'val': 515, 'test': 11}


/content/drive/My Drive/DeepLearning/Pyronear/git/PyroNear/pyronear/datasets/wildfire/wildfire.py:162: Warning: Inconsistent Fire Labeling. Maybe try to label the fire again
Distinct values of ids(304 ≠ 420)
  f" ≠ {wildfire.metadata['fire_id'].max() + 1})", Warning)


In [0]:
def set_seed(seed):
    """Set the seed for pseudo-random number generations
    Args:
        seed (int): seed to set for reproducibility
    """

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def train_batch(model, x, target, optimizer, criterion):
    """Train a model for one iteration
    Args:
        model (torch.nn.Module): model to train
        loader_iter (iter(torch.utils.data.DataLoader)): training dataloader iterator
        optimizer (torch.optim.Optimizer): parameter optimizer
        criterion (torch.nn.Module): criterion object
    Returns:
        batch_loss (float): training loss
    """

    # Forward
    outputs = model(x)

    # Loss computation
    batch_loss = criterion(outputs, target)

    # Backprop
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

    return batch_loss.item()


def train_epoch(model, train_loader, optimizer, criterion, master_bar,
                epoch=0, scheduler=None, device='cpu'):
    """Train a model for one epoch
    Args:
        model (torch.nn.Module): model to train
        train_loader (torch.utils.data.DataLoader): training dataloader
        optimizer (torch.optim.Optimizer): parameter optimizer
        criterion (torch.nn.Module): criterion object
        master_bar (fastprogress.MasterBar): master bar of training progress
        epoch (int): current epoch index
        scheduler (torch.optim._LRScheduler, optional): learning rate scheduler
        device (str): device hosting tensor data
    Returns:
        batch_loss (float): latch batch loss
    """

    # Training
    model.train()
    loader_iter = iter(train_loader)
    train_loss = 0
    for _ in progress_bar(range(len(train_loader)), parent=master_bar):

        x, target = next(loader_iter)
        x, target = x.cuda(), target.type(torch.LongTensor).squeeze().cuda()

        batch_loss = train_batch(model, x, target, optimizer, criterion)
        train_loss += batch_loss
        if scheduler:
            scheduler.step()

        master_bar.child.comment = f"Batch loss: {batch_loss:.4}"

    train_loss /= len(train_loader)

    return train_loss


def evaluate(model, test_loader, criterion, device='cpu'):
    """Evaluation a model on a dataloader
    Args:
        model (torch.nn.Module): model to train
        train_loader (torch.utils.data.DataLoader): validation dataloader
        criterion (torch.nn.Module): criterion object
        device (str): device hosting tensor data
    Returns:
        val_loss (float): validation loss
        acc (float): top1 accuracy
    """
    model.eval()
    val_loss, correct, targets = 0, 0, 0
    with torch.no_grad():
        for x, target in test_loader:
            # Work with tensors on GPU
            #if device.startswith('cuda'):
            x, target = x.cuda(), target.type(torch.LongTensor).squeeze().cuda()

            # Forward + Backward & optimize
            outputs = model.forward(x)
            val_loss += criterion(outputs, target).item()
            # Index of max log-probability
            pred = outputs.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            targets += x.size(0)
    val_loss /= len(test_loader)
    acc = correct / targets

    return val_loss, acc

In [0]:
BATCH_SIZE=32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
# Model definition
model = torchvision.models.resnet18(pretrained=True)

# Change fc
in_features = getattr(model, 'fc').in_features
setattr(model, 'fc', nn.Linear(in_features, 2))
  

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [0]:
model = model.to(device)

In [0]:
epochs=10

lr=0.001
weight_decay=0.001

# optimizer

optimizer = optim.Adam(model.parameters(),
                       betas=(0.9, 0.99),
                       weight_decay=weight_decay)

# Loss function
#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()

# Scheduler
lr_scheduler = OneCycleLR(optimizer, max_lr=lr,epochs=epochs, steps_per_epoch=len(train_loader),cycle_momentum=(not isinstance(optimizer, optim.Adam)))


In [21]:
best_loss = math.inf
mb = master_bar(range(epochs))
for epoch_idx in mb:
    # Training
    # dsTrain.computeSubSet(BATCH_SIZE,dfTrain)
    # train_loader  = data.DataLoader(dsTrain, batch_size=K*BATCH_SIZE, shuffle=False) 

    train_loss=train_epoch(model, train_loader, optimizer, criterion, master_bar=mb, epoch=epoch_idx, scheduler=lr_scheduler, device=device)

    # Evaluation
    val_loss, acc = evaluate(model, val_loader, criterion, device=device)

    mb.comment = f"Epoch {epoch_idx+1}/{epochs}"
    mb.write(f'Epoch {epoch_idx+1}/{epochs} - Training loss: {train_loss:.4} | Validation loss: {val_loss:.4} | Error rate: {1 - acc:.4}')

    # State saving
    if val_loss < best_loss:
        print(f"Validation loss decreased {best_loss:.4} --> {val_loss:.4}: saving state...")
        best_loss = val_loss
      

Validation loss decreased inf --> 0.6783: saving state...
Validation loss decreased 0.6783 --> 0.6348: saving state...
Validation loss decreased 0.6348 --> 0.5223: saving state...
Validation loss decreased 0.5223 --> 0.5159: saving state...
Validation loss decreased 0.5159 --> 0.4356: saving state...


In [0]:
for x, target in train_loader:
  x, target = x.cuda(), target.type(torch.LongTensor).squeeze().cuda()

  # Forward
  outputs = model(x)

  # Loss computation
  batch_loss = criterion(outputs, target)
  break

tensor(0.6990, device='cuda:0', grad_fn=<NllLossBackward>)

torch.Size([64, 2]) torch.Size([64, 1])


tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]])